# Start

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
purchase_data = pd.read_excel("./Analyst_dataset.xlsx", sheet_name='Purchase Exit Survey Data')
airings_data = pd.read_excel("./Analyst_dataset.xlsx", sheet_name='Airings')
lookup_data = pd.read_excel("./Analyst_dataset.xlsx", sheet_name='Lookup', skiprows=1)

# Preprocessing

In [3]:
lookup_data.shape

(44, 3)

In [4]:
lookup_data = lookup_data.dropna(how='all')

In [5]:
lookup_data.shape

(43, 3)

In [6]:
lookup_data['Exit Survey'] = lookup_data['Exit Survey'].str.lower()
lookup_data['Airings'] = lookup_data['Airings'].str.upper()

In [7]:
airings_data['Network'] = airings_data['Network'].str.upper()

# Transposing Purchase Exit Survey Data - Converting dates from columns to rows in Purchase Exit Survey Data

In [8]:
current_year = purchase_data.iloc[0,:].dropna()
current_year = int(current_year)
current_year

2017

In [9]:
months = []
for month in purchase_data.iloc[2,2:].dropna():
    months.append(month)
months

['September', 'October']

In [10]:
# current_month = months[0]
# i = 0
# for count, day in enumerate(day_nums, start=1):
#     if str(day) > str(purchase_data.iloc[3, 2+count]):
#         i += 1
#         current_month = month[i]
#         current_date = str(current_year) + '-' + current_month + '-' + str(day)
#         print(current_date)
#         #print(count)
#     else:
#         current_date = str(current_year) + '-' + current_month + '-' + str(day)
#         print(current_date)
#         #print(count)

day_nums = np.array(purchase_data.iloc[3,2:], dtype=int)

parsed_dates = []
current_month = months[0]
i = 0
for count, today in enumerate(day_nums, start=1):
    try:
        tomorrow = day_nums[count]
    except:
        pass
        # current_date = str(current_year) + '-' + current_month + '-' + str(today)
        # current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        # parsed_dates.append(current_date)
        #print(type(current_date))
    if today > tomorrow:
        current_date = str(current_year) + '-' + current_month + '-' + str(today)
        current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        i += 1
        current_month = months[i]
        parsed_dates.append(current_date)
        #print(current_date)
        #print(count)
    else:
        current_date = str(current_year) + '-' + current_month + '-' + str(today)
        current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        #print(current_date)
        parsed_dates.append(current_date)
        #print(count)

In [11]:
purchase_data.iloc[3,2:] = parsed_dates

In [12]:
purchase_data_transpose = purchase_data.iloc[3:,:].transpose()
#new_df.head()

In [13]:
purchase_data_transpose.index = purchase_data_transpose.iloc[:, 0]

In [14]:
purchase_data_transpose = purchase_data_transpose.iloc[1:]

In [15]:
purchase_data_transpose = purchase_data_transpose.drop(labels=3, axis=1)

In [16]:
purchase_data_transpose.columns = purchase_data_transpose.iloc[0]

In [17]:
purchase_data_transpose = purchase_data_transpose.drop(labels='Source')

In [18]:
purchase_data_transpose.index = pd.to_datetime(purchase_data_transpose.index)

In [19]:
purchase_data_transpose.rename_axis('date', inplace=True)
#purchase_data_transpose

In [20]:
purchase_data_transpose.shape

(56, 30)

# Overall metrics by Network

## Sum of Purchases by Network

In [21]:
sum_of_purchases = purchase_data_transpose.sum(axis=0)
sum_of_purchases = sum_of_purchases.to_frame()
sum_of_purchases = sum_of_purchases.rename(columns={0:'Purchases'})
sum_of_purchases.index = sum_of_purchases.index.str.lower()

In [22]:
sum_of_purchases.shape

(30, 1)

## Joining Purchases to Lookup Data

In [23]:
lookup_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 43
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Exit Survey    43 non-null     object
 1   Airings        38 non-null     object
 2   Exit Survey.1  43 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [24]:
overall_tbl = lookup_data.merge(right=sum_of_purchases, left_on='Exit Survey', right_on='Source', how='left')
overall_tbl.drop(labels='Exit Survey.1', axis=1, inplace=True)
#overall_tbl.rename(columns={0:'Purchases'}, inplace=True)
overall_tbl.set_index('Exit Survey', inplace=True)
overall_tbl.shape

(43, 2)

## Spend and Lift by Network

In [25]:
airings_spend_and_lift = airings_data.groupby('Network')[['Spend', 'Lift']].agg('sum')
airings_spend_and_lift.shape

(19, 2)

## Joining Purchases/Lookup to Spend and Lift

In [26]:
overall_tbl = overall_tbl.merge(right=airings_spend_and_lift,left_on='Airings', right_index=True, how='left')
overall_tbl.shape

(43, 4)

## Computing Metrics by Network

In [27]:
overall_tbl['Conversion Rate'] = overall_tbl['Purchases'] / overall_tbl['Lift'] * 100
overall_tbl['Cost Per Acquisition'] = overall_tbl['Spend'] / overall_tbl['Purchases']
overall_tbl['Cost Per Visitor'] = overall_tbl['Spend'] / overall_tbl['Lift']
overall_tbl['Percent of Purchases'] = overall_tbl['Purchases'] / sum(overall_tbl['Purchases'].fillna(0)) * 100
overall_tbl['Percent of Spend'] = overall_tbl['Spend'] / sum(overall_tbl['Spend'].fillna(0)) * 100
overall_tbl['Percent Pur > Percent Spend'] = overall_tbl['Percent of Purchases'] > overall_tbl['Percent of Spend']
overall_tbl

,Airings,Purchases,Spend,Lift,Conversion Rate,Cost Per Acquisition,Cost Per Visitor,Percent of Purchases,Percent of Spend,Percent Pur > Percent Spend
Exit Survey,,,,,,,,,,
(blank),NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.271186,NaN,False
aapka_colors,NaN,6.0,NaN,NaN,NaN,NaN,NaN,2.542373,NaN,False
baby_first,BABY,1.0,NaN,NaN,NaN,NaN,NaN,0.423729,NaN,False
bloomberg,BLOM,2.0,4966.72,199.0,1.005025,2483.36,24.958392,0.847458,2.242951,False
cbs_sports,CBSS,1.0,NaN,NaN,NaN,NaN,NaN,0.423729,NaN,False
cnbc,CNBC,15.0,6481.42,406.0,3.694581,432.094667,15.964089,6.355932,2.926984,True
cnn,CNN,23.0,18114.35,1081.0,2.12766,787.580435,16.757031,9.745763,8.180369,True
comedy_central,COM,2.0,7501.25,108.0,1.851852,3750.625,69.456019,0.847458,3.387535,False
cozi,COZI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [28]:
overall_tbl.shape

(43, 10)

## Done

# Grouped Metrics by Network and Month

## Purchase Data by Network and Month

In [29]:
purchase_data_by_month = purchase_data_transpose.groupby(pd.Grouper(freq='M')).agg('sum')
purchase_data_by_month

Source,(blank),aapka_colors,baby_first,bloomberg,cbs_sports,cnbc,cnn,comedy_central,dateline,dish_network,...,other,science,sony_entertainment_tv,spectrum_sports,star_plus,television_food_network,the_history_channel,turner_network_tv,willow_tv,zeetv
date,,,,,,,,,,,,,,,,,,,,,
2017-09-30,0.0,1.0,1.0,1.0,1.0,5.0,13.0,0.0,0.0,4.0,...,5.0,1.0,2.0,2.0,2.0,4.0,4.0,2.0,27.0,2.0
2017-10-31,3.0,5.0,0.0,1.0,0.0,10.0,10.0,2.0,10.0,1.0,...,8.0,3.0,2.0,1.0,2.0,0.0,1.0,2.0,29.0,3.0


In [30]:
purchase_data_by_month = purchase_data_by_month.transpose()
purchase_data_by_month = purchase_data_by_month.stack().to_frame()
purchase_data_by_month.rename(columns={0:'Purchases'}, inplace=True)
purchase_data_by_month = purchase_data_by_month.reset_index()
purchase_data_by_month['Source'] = purchase_data_by_month['Source'].str.lower()
purchase_data_by_month

,Source,date,Purchases
0,(blank),2017-09-30,0.0
1,(blank),2017-10-31,3.0
2,aapka_colors,2017-09-30,1.0
3,aapka_colors,2017-10-31,5.0
4,baby_first,2017-09-30,1.0
5,baby_first,2017-10-31,0.0
6,bloomberg,2017-09-30,1.0
7,bloomberg,2017-10-31,1.0
8,cbs_sports,2017-09-30,1.0
9,cbs_sports,2017-10-31,0.0


In [31]:
purchase_data_by_month.shape

(60, 3)

## Airings Sheet

In [32]:
airings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6443 entries, 0 to 6442
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Company       6443 non-null   object        
 1   Date/Time ET  6443 non-null   datetime64[ns]
 2   Rotation      6443 non-null   object        
 3   Creative      6443 non-null   object        
 4   Network       6443 non-null   object        
 5   Spend         6443 non-null   float64       
 6   Lift          6443 non-null   int64         
 7   Program       5729 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 402.8+ KB


In [33]:
# airings_data.groupby([pd.Grouper(key='Date/Time ET', freq='M'), 'Network'])[['Spend', 'Lift']].agg('sum')

## Preparing Lookup Data for Join

In [34]:
lookup_data = lookup_data.drop('Exit Survey.1', axis=1)
# lookup_data = lookup_data.set_index('Exit Survey')
# lookup_data = lookup_data.rename_axis('Source')
lookup_data

,Exit Survey,Airings
0,(blank),NaN
1,aapka_colors,NaN
2,baby_first,BABY
3,bloomberg,BLOM
4,cbs_sports,CBSS
5,cnbc,CNBC
6,cnn,CNN
7,comedy_central,COM
8,cozi,COZI
9,dateline,DATELINE


In [35]:
lookup_data.shape

(43, 2)

## Joining Purchases by network and month to Lookup Data

In [36]:
# joined_tbl = lookup_data[['Exit Survey', 'Airings']].merge(right=purchase_data_by_date, left_on='Exit Survey', right_on='Source', how='left')
# #joined_tbl.drop(labels='Exit Survey', axis=1, inplace=True)
# joined_tbl

In [37]:
# purchase_grouped = purchase_data_by_month.join(lookup_data, how='right')
# purchase_grouped

In [38]:
by_month_tbl = lookup_data.merge(right=purchase_data_by_month, left_on='Exit Survey', right_on='Source', how='left').set_index(['Exit Survey', 'date'])
by_month_tbl

Airings        Source  Purchases
Exit Survey         date                                          
(blank)             2017-09-30        NaN       (blank)        0.0
                    2017-10-31        NaN       (blank)        3.0
aapka_colors        2017-09-30        NaN  aapka_colors        1.0
                    2017-10-31        NaN  aapka_colors        5.0
baby_first          2017-09-30       BABY    baby_first        1.0
...                                   ...           ...        ...
viceland            NaT              VICE           NaN        NaN
destination america NaT              DEST           NaN        NaN
golf                NaT              GOLF           NaN        NaN
tennis              NaT              TENN           NaN        NaN
cnbc world          NaT         CNBCWORLD           NaN        NaN

[73 rows x 3 columns]

In [39]:
print(by_month_tbl.to_string())

                                       Airings                    Source  Purchases
Exit Survey              date                                                      
(blank)                  2017-09-30        NaN                   (blank)        0.0
                         2017-10-31        NaN                   (blank)        3.0
aapka_colors             2017-09-30        NaN              aapka_colors        1.0
                         2017-10-31        NaN              aapka_colors        5.0
baby_first               2017-09-30       BABY                baby_first        1.0
                         2017-10-31       BABY                baby_first        0.0
bloomberg                2017-09-30       BLOM                 bloomberg        1.0
                         2017-10-31       BLOM                 bloomberg        1.0
cbs_sports               2017-09-30       CBSS                cbs_sports        1.0
                         2017-10-31       CBSS                cbs_sports    

In [40]:
by_month_tbl.shape

(73, 3)

## Spend and Lift by Network and Month

In [41]:
# NEED TO drop Network as an index and make a column, join with purchase_grouped and keep the 
airings_spend_lift_grouped = airings_data.groupby(['Network', pd.Grouper(key='Date/Time ET', freq='M')])[['Spend', 'Lift']].agg('sum')
airings_spend_lift_grouped

Spend  Lift
Network   Date/Time ET                
BLOM      2017-09-30     4966.72   199
CNBC      2017-09-30     6481.42   406
CNBCWORLD 2017-10-31     1299.99    51
CNN       2017-09-30     9159.60   507
          2017-10-31     8954.75   574
COM       2017-10-31     7501.25   108
DATELINE  2017-09-30     4211.62   252
          2017-10-31    11076.49   610
DISH      2017-09-30     2513.09   976
          2017-10-31      520.24   116
FS1       2017-09-30     4258.50   183
HIST      2017-09-30     5778.30   253
          2017-10-31     1500.00    77
MSNB      2017-09-30     8091.15   701
          2017-10-31     3238.89   422
NFLN      2017-09-30     5412.80   200
OANN      2017-09-30     4966.80   648
          2017-10-31    19968.20  1724
SCI       2017-09-30     7993.18   132
SPECSPTS  2017-09-30     5580.26   220
STARPLUS  2017-09-30     3441.58   562
          2017-10-31     8338.98  1221
TNT       2017-09-30     5125.50   179
TWC       2017-10-31     7718.71   211
WILO      2017-09-30    19773.49  3138
          2017-10-31    27961.08  5076
ZEETV     2017-09-30    10175.13   922
          2017-10-31    15429.12   819

In [42]:
# airings_spend_lift_grouped.reset_index()

In [43]:
by_month_tbl.reset_index().head()

,Exit Survey,date,Airings,Source,Purchases
0,(blank),2017-09-30,NaN,(blank),0.0
1,(blank),2017-10-31,NaN,(blank),3.0
2,aapka_colors,2017-09-30,NaN,aapka_colors,1.0
3,aapka_colors,2017-10-31,NaN,aapka_colors,5.0
4,baby_first,2017-09-30,BABY,baby_first,1.0


In [44]:
# purchase_grouped.reset_index()

In [45]:
airings_spend_lift_grouped.reset_index().head()

,Network,Date/Time ET,Spend,Lift
0,BLOM,2017-09-30,4966.72,199
1,CNBC,2017-09-30,6481.42,406
2,CNBCWORLD,2017-10-31,1299.99,51
3,CNN,2017-09-30,9159.60,507
4,CNN,2017-10-31,8954.75,574


## Joining Purchases/Lookup to Spend and Lift by Network and Month

In [49]:
# month_and_network_grouped = purchase_grouped.reset_index().merge(right=airings_spend_lift_grouped.reset_index(), left_on=['Airings', 'date'], right_on=['Network', 'Date/Time ET'], how='left')
# month_and_network_grouped

new_tbl = by_month_tbl.reset_index().merge(right=airings_spend_lift_grouped.reset_index(), left_on=['Airings', 'date'], right_on=['Network', 'Date/Time ET'], how='left')
new_tbl.head()

,Exit Survey,date,Airings,Source,Purchases,Network,Date/Time ET,Spend,Lift
0,(blank),2017-09-30,NaN,(blank),0.0,NaN,NaT,NaN,NaN
1,(blank),2017-10-31,NaN,(blank),3.0,NaN,NaT,NaN,NaN
2,aapka_colors,2017-09-30,NaN,aapka_colors,1.0,NaN,NaT,NaN,NaN
3,aapka_colors,2017-10-31,NaN,aapka_colors,5.0,NaN,NaT,NaN,NaN
4,baby_first,2017-09-30,BABY,baby_first,1.0,NaN,NaT,NaN,NaN


In [50]:
new_tbl.shape

(73, 9)

In [51]:
# month_and_network_grouped= month_and_network_grouped.set_index(['Source', 'date']).drop(labels=['Airings', 'Network', 'Date/Time ET'], axis=1)
# month_and_network_grouped

new_tbl = new_tbl.set_index(['Exit Survey', 'date']).drop(labels=['Airings', 'Network', 'Date/Time ET', 'Source'], axis=1)
new_tbl

Purchases  Spend  Lift
Exit Survey         date                              
(blank)             2017-09-30        0.0    NaN   NaN
                    2017-10-31        3.0    NaN   NaN
aapka_colors        2017-09-30        1.0    NaN   NaN
                    2017-10-31        5.0    NaN   NaN
baby_first          2017-09-30        1.0    NaN   NaN
...                                   ...    ...   ...
viceland            NaT               NaN    NaN   NaN
destination america NaT               NaN    NaN   NaN
golf                NaT               NaN    NaN   NaN
tennis              NaT               NaN    NaN   NaN
cnbc world          NaT               NaN    NaN   NaN

[73 rows x 3 columns]

In [52]:
new_tbl.shape

(73, 3)

In [53]:
print(new_tbl.to_string())

                                     Purchases     Spend    Lift
Exit Survey              date                                   
(blank)                  2017-09-30        0.0       NaN     NaN
                         2017-10-31        3.0       NaN     NaN
aapka_colors             2017-09-30        1.0       NaN     NaN
                         2017-10-31        5.0       NaN     NaN
baby_first               2017-09-30        1.0       NaN     NaN
                         2017-10-31        0.0       NaN     NaN
bloomberg                2017-09-30        1.0   4966.72   199.0
                         2017-10-31        1.0       NaN     NaN
cbs_sports               2017-09-30        1.0       NaN     NaN
                         2017-10-31        0.0       NaN     NaN
cnbc                     2017-09-30        5.0   6481.42   406.0
                         2017-10-31       10.0       NaN     NaN
cnn                      2017-09-30       13.0   9159.60   507.0
                         

In [54]:
new_tbl['Conversion Rate'] = new_tbl['Purchases'] / new_tbl['Lift'] * 100
new_tbl['Cost Per Acquisition'] = new_tbl['Spend'] / new_tbl['Purchases']
new_tbl['Cost Per Visitor'] = new_tbl['Spend'] / new_tbl['Lift']
new_tbl['Percent of Purchases'] = new_tbl['Purchases'] / sum(new_tbl['Purchases'].fillna(0)) * 100
new_tbl['Percent of Spend'] = new_tbl['Spend'] / sum(new_tbl['Spend'].fillna(0)) * 100
new_tbl['Percent Pur > Percent Spend'] = new_tbl['Percent of Purchases'] > new_tbl['Percent of Spend']
new_tbl

Purchases  Spend  Lift  Conversion Rate  \
Exit Survey         date                                                  
(blank)             2017-09-30        0.0    NaN   NaN              NaN   
                    2017-10-31        3.0    NaN   NaN              NaN   
aapka_colors        2017-09-30        1.0    NaN   NaN              NaN   
                    2017-10-31        5.0    NaN   NaN              NaN   
baby_first          2017-09-30        1.0    NaN   NaN              NaN   
...                                   ...    ...   ...              ...   
viceland            NaT               NaN    NaN   NaN              NaN   
destination america NaT               NaN    NaN   NaN              NaN   
golf                NaT               NaN    NaN   NaN              NaN   
tennis              NaT               NaN    NaN   NaN              NaN   
cnbc world          NaT               NaN    NaN   NaN              NaN   

                                Cost Per Acquisition  Cost Per Visitor  \
Exit Survey         date                                                 
(blank)             2017-09-30                   NaN               NaN   
                    2017-10-31                   NaN               NaN   
aapka_colors        2017-09-30                   NaN               NaN   
                    2017-10-31                   NaN               NaN   
baby_first          2017-09-30                   NaN               NaN   
...                                              ...               ...   
viceland            NaT                          NaN               NaN   
destination america NaT                          NaN               NaN   
golf                NaT                          NaN               NaN   
tennis              NaT                          NaN               NaN   
cnbc world          NaT                          NaN               NaN   

                                Percent of Purchases  Percent of Spend  \
Exit Survey         date                                                 
(blank)             2017-09-30              0.000000               NaN   
                    2017-10-31              1.271186               NaN   
aapka_colors        2017-09-30              0.423729               NaN   
                    2017-10-31              2.118644               NaN   
baby_first          2017-09-30              0.423729               NaN   
...                                              ...               ...   
viceland            NaT                          NaN               NaN   
destination america NaT                          NaN               NaN   
golf                NaT                          NaN               NaN   
tennis              NaT                          NaN               NaN   
cnbc world          NaT                          NaN               NaN   

                                Percent Pur > Percent Spend  
Exit Survey         date                                     
(blank)             2017-09-30                        False  
                    2017-10-31                        False  
aapka_colors        2017-09-30                        False  
                    2017-10-31                        False  
baby_first          2017-09-30                        False  
...                                                     ...  
viceland            NaT                               False  
destination america NaT                               False  
golf                NaT                               False  
tennis              NaT                               False  
cnbc world          NaT                               False  

[73 rows x 9 columns]

In [55]:
print(new_tbl.to_string())

                                     Purchases     Spend    Lift  Conversion Rate  Cost Per Acquisition  Cost Per Visitor  Percent of Purchases  Percent of Spend  Percent Pur > Percent Spend
Exit Survey              date                                                                                                                                                                 
(blank)                  2017-09-30        0.0       NaN     NaN              NaN                   NaN               NaN              0.000000               NaN                        False
                         2017-10-31        3.0       NaN     NaN              NaN                   NaN               NaN              1.271186               NaN                        False
aapka_colors             2017-09-30        1.0       NaN     NaN              NaN                   NaN               NaN              0.423729               NaN                        False
                         2017-10-31        5.

## Done

In [ ]:
airings_data.query('Network == "FOOD"')

# What networks have purchases but no spend?

In [ ]:
airings_data.query('Spend == 0')['Network'].value_counts()

In [ ]:
airings_data.groupby('Network')[['Spend', 'Lift']].agg('sum')